## **Instalamos las dependencias necesarias**

*(ultralytics no esta por defecto así que toca descargarlo a través de PIP)*

In [ ]:
!pip install ultralytics

from ultralytics import YOLO
import cv2

## **Cargamos nuestro Drive donde tendremos nuestro YAML con las configuraciones y nuestro DATASET para entrenar**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## *(Opcional)* **Utilizar ffmpeg para crear el DATASET de imágenes y separarlos para entrenamiento y prueba**

In [ ]:
!apt-get install ffmpeg

!ffmpeg -i /content/drive/MyDrive/dev/dataset/test-video.mp4 -vf "fps=5" /content/fotogramas/frame_%04d.jpg

In [ ]:
import os
import random
import shutil

# Directorios de origen y destino
source_dir = "/content/fotogramas"
train_dir = "/content/fotogramas/train"
val_dir = "/content/fotogramas/val"

# Crear directorios si no existen
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Obtener la lista de archivos de imagen
image_files = [f for f in os.listdir(source_dir) if f.startswith("frame_") and f.endswith(".jpg")]

# Calcular el número de imágenes para el conjunto de entrenamiento
num_train = int(0.8 * len(image_files))

# Mezclar aleatoriamente la lista de archivos
random.shuffle(image_files)

# Dividir los archivos en conjuntos de entrenamiento y validación
train_files = image_files[:num_train]
val_files = image_files[num_train:]

# Mover los archivos a los directorios correspondientes
for file in train_files:
  source_path = os.path.join(source_dir, file)
  destination_path = os.path.join(train_dir, file)
  shutil.move(source_path, destination_path)

for file in val_files:
  source_path = os.path.join(source_dir, file)
  destination_path = os.path.join(val_dir, file)
  shutil.move(source_path, destination_path)

## **Cargamos un modelo BASE de YOLO para luego empezar con el entrenamiento**

Recordar que el **ball_data.yaml** tiene informacion primordial para el entrenamiento

```
path: /content/drive/MyDrive/dev/dataset
train: train/images
val: val/images

nc: 1
names: ['sports ball']

```
En este caso todo el contenido de entrenamiento está dentro de mi cuenta de Google Drive en: **/dev/dataset**, pero en Colab seria **/content/drive/MyDrive/dev/dataset** una vez montada la unidad de Drive en Colab


In [ ]:
model = YOLO('yolov8m.pt') # yolov8n.pt | yolov8s.pt | yolov8m.pt

model.train(
    data='/content/drive/MyDrive/dev/dataset/ball_data.yaml',
    epochs=50,                          # Número de épocas
    batch=16,                           # Tamaño del lote
    imgsz=720,                          # Tamaño de las imágenes
    project='sports_tracking',          # Carpeta donde se guardará el proyecto
    name='yolo_ball_tracking',          # Nombre del experimento
    pretrained=True                     # Usar pesos preentrenados
)

## **Probamos el modelo recién entrenado** *(o cargamos uno ya pre-entrenado)*

In [ ]:
model = YOLO('/content/sports_tracking/yolo_ball_tracking/weights/best.pt') # Aquí tenemos que señalar la ubicación de nuestro modelo recién entrenado o pre-entrenado para probarlo

results = model.predict(
    source='/content/drive/MyDrive/dev/dataset/test-video.mp4',
    conf=0.4, # Umbral de confianza
    save=True,
    save_txt=True
)

## **Probamos de forma visual** *(con imagenes)* **el resultado del entrenamiento**

In [ ]:
import cv2
from ultralytics import YOLO
from google.colab.patches import cv2_imshow
from IPython.display import clear_output

model = YOLO('/content/sports_tracking/yolo_ball_tracking/weights/best.pt')

cap = cv2.VideoCapture('/content/drive/MyDrive/dev/dataset/test-video.mp4')
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Realizar predicciones
    results = model(frame, conf=0.4) # Realizar predicción con el modelo

    # Dibujar las detecciones
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0]) # Coordenadas
            confidence = box.conf[0]  # Confianza
            label = f"ball {confidence:.2f}" # Etiqueta

            # Dibujar el rectángulo y la etiqueta
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2_imshow(frame)

cap.release()
cv2.destroyAllWindows()